# Solving the Frozen Lake Problem with Value Iteration

In the previous chapter, we have learned about the frozen lake environment. The frozen
lake environment is shown below:


![title](Images/4.png)

Let's recap the frozen lake environment a bit. In the frozen lake environment shown above,
the following applies:
    
* S implies the starting state
* F implies the frozen states
* H implies the hold states
* G implies the goal state

We learned that in the frozen lake environment, our goal is to reach the goal state G from
the starting state S without visiting the hole states H. That is, while trying to reach the goal
state G from the starting state S if the agent visits the hole state H then it will fall into the
hole and die as shown below:


![title](Images/5.png)

So, the goal of the agent is to reach the state G starting from the state S without visiting the
hole states H as shown below:


![title](Images/6.png)

How can we achieve this goal? That is, how can we reach the state G from S without
visiting H? We learned that the optimal policy tells the agent to perform correct action in
each state. So, if we find the optimal policy then we can reach the state G from S visiting the state H. Okay, how to find the optimal policy? We can use the value iteration method
we just learned to find the optimal policy.


Remember that all our states (S to G) will be encoded from 0 to 16 and all the four actions -
left, down, up, right will be encoded from 0 to 3 in the gym toolkit.
So, in this section, we will learn how to find the optimal policy using the value iteration
method so that the agent can reach the state G from S without visiting H.

Remember to install all the necessary libraries beforehand:

In [16]:
#Install pyton libraries

#pip intall gym==0.26.2
#pip intall numpy

First, let's import the necessary libraries:

In [17]:
import gym
import numpy as np
import time

Now, let's create the frozen lake environment using gym:

In [18]:
slipperyState = False

In [19]:
env = gym.make('FrozenLake-v1', render_mode = "human", is_slippery=slipperyState)
env.reset()

(0, {'prob': 1})

Let's look at the frozen lake environment using the render function:

In [20]:
env.render()

As we can notice, our agent is in the state S and it has to reach the state G without visiting
the states H. So, let's learn how to compute the optimal policy using the value iteration
method.

Create an empty Q Table

In [21]:
#BEFORE TRAINING Q TABLE
#Check action size and atate size
action_size = env.action_space.n
state_size = env.observation_space.n   
print('action size=', action_size, 'state size=', state_size)

# Start with very small values for all our Q(s,a)
q_table = np.zeros([state_size, action_size])
print('Q Table Init = \n', q_table, '\nQ Table Shape = ', q_table.shape)

action size= 4 state size= 16
Q Table Init = 
 [[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]] 
Q Table Shape =  (16, 4)


First, let's learn how to compute the optimal value function and then we will see how to
extract the optimal policy from the computed optimal value function.


## Computing optimal value function

We will define a function called `value_iteration` where we compute the optimal value
function iteratively by taking maximum over Q function. For
better understanding, let's closely look at the every line of the function and then we look at
the complete function at the end which gives us more clarity.



Define `value_iteration` function which takes the environment as a parameter:
    
    

In [22]:
def value_iteration(env):

    #set the number of iterations
    num_iterations = 1000

    #set the threshold number for checking the convergence of the value function
    threshold = 1e-20

    #we also set the discount factor
    gamma = 0.95

    #now, we will initialize the value table, with the value of all states to zero
    value_table = np.zeros(env.observation_space.n)

    #for every iteration
    for i in range(num_iterations):

        #update the value table, that is, we learned that on every iteration, we use the updated value
        #table (state values) from the previous iteration
        updated_value_table = np.copy(value_table)

        #now, we compute the value function (state value) by taking the maximum of Q value.

        #thus, for each state, we compute the Q values of all the actions in the state and then
        #we update the value of the state as the one which has maximum Q value as shown below:
        for s in range(env.observation_space.n):

            Q_values = [sum([prob*(r + gamma * updated_value_table[s_])
                             for prob, s_, r, _ in env.P[s][a]])
                                   for a in range(env.action_space.n)]
        
            #print(Q_values)
        
            value_table[s] = max(Q_values)
            
            
  
            # Update Q Table
            for i, qValue in enumerate(Q_values):
                q_table[s,i] = qValue
            

        #after computing the value table, that is, value of all the states, we check whether the
        #difference between value table obtained in the current iteration and previous iteration is
        #less than or equal to a threshold value if it is less then we break the loop and return the
        #value table as our optimal value function as shown below:

        if (np.sum(np.fabs(updated_value_table - value_table)) <= threshold):
             break

    return value_table

Now, that we have computed the optimal value function by taking the maximum over Q
values, let's see how to extract the optimal policy from the optimal value function.


## Extracting optimal policy from the optimal value function

In the previous step, we computed the optimal value function. Now, let see how to extract
the optimal policy from the computed optimal value function.


First, we define a function called `extract_policy` which takes the `value_table` as a
parameter:


In [23]:
def extract_policy(value_table):

    #set the discount factor
    gamma = 0.95

    #first, we initialize the policy with zeros, that is, first, we set the actions for all the states to
    #be zero
    policy = np.zeros(env.observation_space.n)

    #now, we compute the Q function using the optimal value function obtained from the
    #previous step. After computing the Q function, we can extract policy by selecting action which has
    #maximum Q value. Since we are computing the Q function using the optimal value
    #function, the policy extracted from the Q function will be the optimal policy.

    #As shown below, for each state, we compute the Q values for all the actions in the state and
    #then we extract policy by selecting the action which has maximum Q value.

    #for each state
    for s in range(env.observation_space.n):

        #compute the Q value of all the actions in the state
        Q_values = [sum([prob*(r + gamma * value_table[s_])
                             for prob, s_, r, _ in env.P[s][a]])
                                   for a in range(env.action_space.n)]

        #extract policy by selecting the action which has maximum Q value
        policy[s] = np.argmax(np.array(Q_values))

    return policy

That's it! Now, we will see how to extract the optimal policy in our frozen lake
environment.

## Putting it all together
We learn that in the frozen lake environment our goal is to find the optimal policy which
selects the correct action in each state so that we can reach the state G from the state
A without visiting the hole states.

First, we compute the optimal value function using our `value_iteration` function by
passing our frozen lake environment as the parameter:


In [24]:
value_iteration(env=env)

array([0.77378094, 0.81450625, 0.857375  , 0.81450625, 0.81450625,
       0.        , 0.9025    , 0.        , 0.857375  , 0.9025    ,
       0.95      , 0.        , 0.        , 0.95      , 1.        ,
       0.        ])

Now we can see what the Q Table looks like after training

In [25]:
print('Q Table = \n', q_table)

Q Table = 
 [[0.73509189 0.77378094 0.77378094 0.73509189]
 [0.73509189 0.         0.81450625 0.77378094]
 [0.77378094 0.857375   0.77378094 0.81450625]
 [0.81450625 0.         0.77378094 0.77378094]
 [0.77378094 0.81450625 0.         0.73509189]
 [0.         0.         0.         0.        ]
 [0.         0.9025     0.         0.81450625]
 [0.         0.         0.         0.        ]
 [0.81450625 0.         0.857375   0.77378094]
 [0.81450625 0.9025     0.9025     0.        ]
 [0.857375   0.95       0.         0.857375  ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.9025     0.95       0.857375  ]
 [0.9025     0.95       1.         0.9025    ]
 [0.         0.         0.         0.        ]]


Next we can get the optimal value

In [26]:
optimal_value_function = value_iteration(env=env)

In [27]:
optimal_value_function

array([0.77378094, 0.81450625, 0.857375  , 0.81450625, 0.81450625,
       0.        , 0.9025    , 0.        , 0.857375  , 0.9025    ,
       0.95      , 0.        , 0.        , 0.95      , 1.        ,
       0.        ])

Next, we extract the optimal policy from the optimal value function using our
extract_policy function as shown below:

In [28]:
optimal_policy = extract_policy(optimal_value_function)

We can print the obtained optimal policy:

In [29]:
print(optimal_policy)

[1. 2. 1. 0. 1. 0. 1. 0. 2. 1. 1. 0. 0. 2. 2. 0.]


As we can observe, our optimal policy tells us to
perform the correct action in each state.

Now, that we have learned what is value iteration and how to perform the value iteration
method to compute the optimal policy in our frozen lake environment, in the next section
we will learn about another interesting method called the policy iteration.

Finally we can use the optimal policy Q table in the Forzen lake environment

In [15]:
env=gym.make("FrozenLake-v1", render_mode='human', is_slippery=slipperyState)
env.reset()
state = env.reset()[0]
time.sleep(1)
for steps in range(100):
    env.render()
    action = np.argmax(q_table[state, :])
    state, reward, done, done1, info = env.step(action)
    if done:
        break
env.close()

C:\Users\Alan Chow\anaconda3\envs\Reinforcement_learning\lib\site-packages\gym\utils\passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):
